In [9]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist  
import math
from numpy import random

def sigmoid(x):
    return 1/(1+np.exp(-x))

def derivative_sigmoid(x):
    return np.exp(x)/(1+np.exp(x))**2

def softmax(x):
    total = 0
    output = [0.0]*len(x)
    for index in range(len(x)):
        output[index] = np.exp(x[index])
        total = total + output[index]
    output = output/total
    return output
    
def derivative_softmax(x):
    return np.multiply(softmax(x), [1]*len(x) - softmax(x))

def ReLu(x):
    for i in range(len(x)):
        for j in range(len(x[i])):
            if(x[i][j] < 0 or x[i][j] == 0):
                x[i][j] = 0
    return x
def derivative_ReLu(x):
    for i in range(len(x)):
        for j in range(len(x[i])):
            if(x[i][j] < 0 or x[i][j] == 0):
                x[i][j] = 0
            else:
                x[i][j] = 1
    return x

def LeakyReLu(x, a):
    x = np.array(x)
    y = x * (x > 0) + a * x * (x < 0)
    return y

def derivative_LeakyReLu(x, a):
    x = np.array(x)
    y = (x > 0) + a * (x < 0)
    ##for i in range(len(x)):
        ##for j in range(len(x[i])):
            ##if(x[i][j] < 0 or x[i][j] == 0):
                ##y[i][j] = a
            ##else:
                ##y[i][j] = 1 
    return y

def Batch_Normalization(activation_pre, theta, beta):
    activation_post = []
    activation_hat = []
    mu = np.mean(activation_pre, axis=0)
    sigma_2 = np.var(activation_pre, axis=0)
    for i in range(len(activation_pre)):
        activation_hat.append((activation_pre[i] - mu)/np.sqrt(sigma_2 + 10e-8))
        activation_post.append(np.multiply(theta, (activation_pre[i] - mu)/np.sqrt(sigma_2 + 10e-8)) + beta)
    return (np.array(activation_post), np.array(activation_hat), mu, sigma_2)

def Batch_Normalization_Derivative(dL_activation_post, activation_hat, activation_pre, mu, sigma_2, theta, beta, getLinearDerivatives):
    if(getLinearDerivatives == True):
        dL_theta = np.zeros_like(theta)
        for i in range(Batch_Size):
            dL_theta = dL_theta + np.multiply(dL_activation_post[i], activation_hat[i])
        dL_beta = np.sum(dL_activation_post, axis=0)
        dL_activation_hat = []
        for j in range(Batch_Size):
            dL_activation_hat.append(np.multiply(dL_activation_post[j], theta))
        dL_mu = np.zeros_like(mu)
        for k in range(Batch_Size):
            dL_mu = dL_mu + np.multiply(dL_activation_hat[k], -1/np.sqrt(sigma_2 + 10e-8))
        dL_sigma_2 = np.zeros_like(sigma_2)
        for l in range(Batch_Size):
            dL_sigma_2 = dL_sigma_2 + np.multiply(dL_activation_hat[l], -1/2*(activation_pre[l] - mu)/(np.sqrt(sigma_2 + 10e-8))**3)
        dL_activation_pre = []
        for m in range(Batch_Size):
            dL_activation_pre.append(np.multiply(dL_activation_hat[m], 1/np.sqrt(sigma_2 + 10e-8)) + 1/Batch_Size * dL_mu + np.multiply(dL_sigma_2, 2/Batch_Size * (activation_pre[m] - mu)))
        return (dL_activation_pre, dL_sigma_2, dL_beta)
    elif(getLinearDerivatives == False):
        dL_activation_hat = []
        for j in range(Batch_Size):
            dL_activation_hat.append(np.multiply(dL_activation_post[j], theta))
        dL_mu = np.zeros_like(mu)
        for k in range(Batch_Size):
            dL_mu = dL_mu + np.multiply(dL_activation_hat[k], -1/np.sqrt(sigma_2 + 10e-8))
        dL_sigma_2 = np.zeros_like(sigma_2)
        for l in range(Batch_Size):
            dL_sigma_2 = dL_sigma_2 + np.multiply(dL_activation_hat[l], -1/2*(activation_pre[l] - mu)/(np.sqrt(sigma_2 + 10e-8))**3)
        dL_activation_pre = []
        for m in range(Batch_Size):
            dL_activation_pre.append(np.multiply(dL_activation_hat[m], 1/np.sqrt(sigma_2 + 10e-8)) + 1/Batch_Size * dL_mu + np.multiply(dL_sigma_2, 2/Batch_Size * (activation_pre[m] - mu)))
        return dL_activation_pre

def test_Batch_Normalization(activation_pre, true_mu, true_sigma_2, theta, beta):
    activation_post = []
    for i in range(len(activation_pre)):
        activation_post.append(np.multiply(theta, (activation_pre[i] - true_mu)/np.sqrt(true_sigma_2 + 10e-8)) + beta)
    return activation_post 

def Spectral_Normalization(matrix):
    u, s, vh = np.linalg.svd(matrix, full_matrices=True)
    return matrix/s[0]

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

Whole_Training_Set = X_train

##for i in range(len(X_train)):
    ##if(Y_train[i] == 5):
        ##Whole_Training_Set.append(X_train[i])
##for i in range(len(X_test)):
    ##if(Y_test[i] == 5):
        ##Whole_Training_Set.append(X_test[i])

Whole_Training_Set = np.array(Whole_Training_Set)

Batch_Size = 50

##Generator Weights/Biases here 

Weights_0_to_1_G = np.random.normal(0, math.sqrt(2/100), 128*100).reshape(128, 100)
theta_1_G = np.ones(128)
beta_1_G = np.zeros(128)

Weights_1_to_2_G = np.random.normal(0, math.sqrt(2/128), 256*128).reshape(256, 128)
theta_2_G = np.ones(256)
beta_2_G = np.zeros(256)

Weights_2_to_3_G = np.random.normal(0, math.sqrt(2/256), 512*256).reshape(512, 256)
theta_3_G = np.ones(512)
beta_3_G = np.zeros(512)

Weights_3_to_4_G = np.random.normal(0, math.sqrt(2/512), 1024*512).reshape(1024, 512)
theta_4_G = np.ones(1024)
beta_4_G = np.zeros(1024)

Weights_4_to_5_G = np.random.normal(0, math.sqrt(2/1024), 784*1024).reshape(784, 1024)
bias_5_G = np.random.normal(0, math.sqrt(2/1024), 784)

##Discriminator Weights/Biases here

Weights_0_to_1_D = np.random.normal(0, math.sqrt(2/784), 628*784).reshape(628, 784)
beta_1_D = np.random.normal(0, math.sqrt(2/784), 628)

Weights_1_to_2_D = np.random.normal(0, math.sqrt(2/628), 471*628).reshape(471, 628)
beta_2_D = np.random.normal(0, math.sqrt(2/628), 471)

Weights_2_to_3_D = np.random.normal(0, math.sqrt(2/471), 314*471).reshape(314, 471)
beta_3_D = np.random.normal(0, math.sqrt(2/471), 314)

Weights_3_to_4_D = np.random.normal(0, math.sqrt(2/314), 157*314).reshape(157, 314)
beta_4_D = np.random.normal(0, math.sqrt(2/314), 157)

Weights_4_to_5_D = np.random.normal(0, math.sqrt(2/157), 157)
bias_5_D = np.random.normal(0, math.sqrt(2/157))

##Adam matrices for Generator 

Weights_0_to_1_G_m_matrix = np.zeros_like(Weights_0_to_1_G)
Weights_0_to_1_G_v_matrix = np.zeros_like(Weights_0_to_1_G)
theta_1_G_m_matrix = np.zeros_like(theta_1_G)
theta_1_G_v_matrix = np.zeros_like(theta_1_G)
beta_1_G_m_matrix = np.zeros_like(beta_1_G)
beta_1_G_v_matrix = np.zeros_like(beta_1_G)

Weights_1_to_2_G_m_matrix = np.zeros_like(Weights_1_to_2_G)
Weights_1_to_2_G_v_matrix = np.zeros_like(Weights_1_to_2_G)
theta_2_G_m_matrix = np.zeros_like(theta_2_G)
theta_2_G_v_matrix = np.zeros_like(theta_2_G)
beta_2_G_m_matrix = np.zeros_like(beta_2_G)
beta_2_G_v_matrix = np.zeros_like(beta_2_G)

Weights_2_to_3_G_m_matrix = np.zeros_like(Weights_2_to_3_G)
Weights_2_to_3_G_v_matrix = np.zeros_like(Weights_2_to_3_G)
theta_3_G_m_matrix = np.zeros_like(theta_3_G)
theta_3_G_v_matrix = np.zeros_like(theta_3_G)
beta_3_G_m_matrix = np.zeros_like(beta_3_G)
beta_3_G_v_matrix = np.zeros_like(beta_3_G)

Weights_3_to_4_G_m_matrix = np.zeros_like(Weights_3_to_4_G)
Weights_3_to_4_G_v_matrix = np.zeros_like(Weights_3_to_4_G)
theta_4_G_m_matrix = np.zeros_like(theta_4_G)
theta_4_G_v_matrix = np.zeros_like(theta_4_G)
beta_4_G_m_matrix = np.zeros_like(beta_4_G)
beta_4_G_v_matrix = np.zeros_like(beta_4_G)

Weights_4_to_5_G_m_matrix = np.zeros_like(Weights_4_to_5_G)
Weights_4_to_5_G_v_matrix = np.zeros_like(Weights_4_to_5_G)
bias_5_G_m_matrix = np.zeros_like(bias_5_G)
bias_5_G_v_matrix = np.zeros_like(bias_5_G)

##Adam matrices for Discriminator 

Weights_0_to_1_D_m_matrix = np.zeros_like(Weights_0_to_1_D)
Weights_0_to_1_D_v_matrix = np.zeros_like(Weights_0_to_1_D)
beta_1_D_m_matrix = np.zeros_like(beta_1_D)
beta_1_D_v_matrix = np.zeros_like(beta_1_D)

Weights_1_to_2_D_m_matrix = np.zeros_like(Weights_1_to_2_D)
Weights_1_to_2_D_v_matrix = np.zeros_like(Weights_1_to_2_D)
beta_2_D_m_matrix = np.zeros_like(beta_2_D)
beta_2_D_v_matrix = np.zeros_like(beta_2_D)

Weights_2_to_3_D_m_matrix = np.zeros_like(Weights_2_to_3_D)
Weights_2_to_3_D_v_matrix = np.zeros_like(Weights_2_to_3_D)
beta_3_D_m_matrix = np.zeros_like(beta_3_D)
beta_3_D_v_matrix = np.zeros_like(beta_3_D)

Weights_3_to_4_D_m_matrix = np.zeros_like(Weights_3_to_4_D)
Weights_3_to_4_D_v_matrix = np.zeros_like(Weights_3_to_4_D)
beta_4_D_m_matrix = np.zeros_like(beta_4_D)
beta_4_D_v_matrix = np.zeros_like(beta_4_D)

Weights_4_to_5_D_m_matrix = np.zeros_like(Weights_4_to_5_D)
Weights_4_to_5_D_v_matrix = np.zeros_like(Weights_4_to_5_D)
bias_5_D_m_matrix = np.zeros_like(bias_5_D)
bias_5_D_v_matrix = np.zeros_like(bias_5_D)

mu_1_G_true = np.zeros(128)
sigma_2_1_G_true = np.zeros(128)
mu_2_G_true = np.zeros(256)
sigma_2_2_G_true = np.zeros(256)
mu_3_G_true = np.zeros(512)
sigma_2_3_G_true = np.zeros(512)
mu_4_G_true = np.zeros(1024)
sigma_2_4_G_true = np.zeros(1024)

c = 0

for e in range(0, 10):

    ##indices = np.random.choice(len(Whole_Training_Set), size=6000, replace=False)
    Generation_Training_set = Whole_Training_Set##[indices,:]
    np.random.shuffle(Generation_Training_set)
    Generation_Training_set = Generation_Training_set.reshape(int(60000/Batch_Size), Batch_Size, 784)

    for r in range(1, len(Generation_Training_set) + 1):

        c+=1

        latent_space_random_vector = np.random.normal(0, 1, 100*Batch_Size).reshape(Batch_Size, 100)

        ##We generate our 5's here

        z_1_G_pre = [] 
        for i in range(Batch_Size):
            z_1_G_pre.append(np.matmul(Weights_0_to_1_G, latent_space_random_vector[i]))
        activation_1_G_pre = LeakyReLu(z_1_G_pre, 0.2)
        (activation_1_G_post, activation_1_G_hat, mu_1_G, sigma_2_1_G) = Batch_Normalization(activation_1_G_pre, theta_1_G, beta_1_G)

        mu_1_G_true = 0.1 * mu_1_G + (1-0.1) * mu_1_G_true
        sigma_2_1_G_true = 0.1 * sigma_2_1_G + (1-0.1) * sigma_2_1_G_true

        z_2_G_pre = []
        for j in range(Batch_Size):
            z_2_G_pre.append(np.matmul(Weights_1_to_2_G, activation_1_G_post[j]))
        activation_2_G_pre = LeakyReLu(z_2_G_pre, 0.2)
        (activation_2_G_post, activation_2_G_hat, mu_2_G, sigma_2_2_G) = Batch_Normalization(activation_2_G_pre, theta_2_G, beta_2_G)

        mu_2_G_true = 0.1 * mu_2_G + (1-0.1) * mu_2_G_true
        sigma_2_2_G_true = 0.1 * sigma_2_2_G + (1-0.1) * sigma_2_2_G_true

        z_3_G_pre = []
        for k in range(Batch_Size):
            z_3_G_pre.append(np.matmul(Weights_2_to_3_G, activation_2_G_post[k]))
        activation_3_G_pre = LeakyReLu(z_3_G_pre, 0.2)
        (activation_3_G_post, activation_3_G_hat, mu_3_G, sigma_2_3_G) = Batch_Normalization(activation_3_G_pre, theta_3_G, beta_3_G)

        mu_3_G_true = 0.1 * mu_3_G + (1-0.1) * mu_3_G_true
        sigma_2_3_G_true = 0.1 * sigma_2_3_G + (1-0.1) * sigma_2_3_G_true

        z_4_G_pre = []
        for k in range(Batch_Size):
            z_4_G_pre.append(np.matmul(Weights_3_to_4_G, activation_3_G_post[k]))
        activation_4_G_pre = LeakyReLu(z_4_G_pre, 0.2)
        (activation_4_G_post, activation_4_G_hat, mu_4_G, sigma_2_4_G) = Batch_Normalization(activation_4_G_pre, theta_4_G, beta_4_G)

        mu_4_G_true = 0.1 * mu_4_G + (1-0.1) * mu_4_G_true
        sigma_2_4_G_true = 0.1 * sigma_2_4_G + (1-0.1) * sigma_2_4_G_true

        z_5_G = []
        for l in range(Batch_Size):
            z_5_G.append(np.matmul(Weights_4_to_5_G, activation_4_G_post[l]) + bias_5_G)
        activation_5_G = np.tanh(z_5_G) 

        Transformed_Generation_Training_Set = (Generation_Training_set[r-1] + np.ones_like(Generation_Training_set[r-1]))/128.5 - np.ones_like(Generation_Training_set[r-1])        

        noise = np.random.normal(0, 1 - (c-1)/(10*len(Generation_Training_set) - 1), 784*Batch_Size).reshape(Batch_Size, 784)
        bruh = np.random.binomial(1, 1/2, 1)

        if(bruh == 0):
            activation_5_G = 128.5 * (activation_5_G + np.ones_like(activation_5_G)) - np.ones_like(activation_5_G) + noise
            activation_5_G = (activation_5_G + np.ones_like(activation_5_G))/128.5 - np.ones_like(activation_5_G)  
        else:
            Transformed_Generation_Training_Set = (Generation_Training_set[r-1] + noise + np.ones_like(Generation_Training_set[r-1]))/128.5 - np.ones_like(Generation_Training_set[r-1])

        ##We feed our Generated 5's through the discriminator here 

        z_1_D_G = []
        for i in range(Batch_Size):
            z_1_D_G.append(np.matmul(Weights_0_to_1_D, activation_5_G[i]) + beta_1_D)
        activation_1_D_G = LeakyReLu(z_1_D_G, 0.2)

        z_2_D_G = []
        for j in range(Batch_Size):
            z_2_D_G.append(np.matmul(Weights_1_to_2_D, activation_1_D_G[j]) + beta_2_D)
        activation_2_D_G = LeakyReLu(z_2_D_G, 0.2)

        z_3_D_G = []
        for k in range(Batch_Size):
            z_3_D_G.append(np.matmul(Weights_2_to_3_D, activation_2_D_G[k]) + beta_3_D)
        activation_3_D_G = LeakyReLu(z_3_D_G, 0.2)

        z_4_D_G = []
        for k in range(Batch_Size):
            z_4_D_G.append(np.matmul(Weights_3_to_4_D, activation_3_D_G[k]) + beta_4_D)
        activation_4_D_G = LeakyReLu(z_4_D_G, 0.2)
 
        z_5_D_G = []
        for l in range(Batch_Size):
            z_5_D_G.append(np.dot(Weights_4_to_5_D, activation_4_D_G[l]) + bias_5_D)
        activation_5_D_G = sigmoid(np.array(z_5_D_G))

        ##We feed our sampled 5's through the discriminator here 

        z_1_D_S = []
        for i in range(Batch_Size):
            z_1_D_S.append(np.matmul(Weights_0_to_1_D, Transformed_Generation_Training_Set[i]) + beta_1_D)
        activation_1_D_S = LeakyReLu(z_1_D_S, 0.2)

        z_2_D_S = []
        for j in range(Batch_Size):
            z_2_D_S.append(np.matmul(Weights_1_to_2_D, activation_1_D_S[j]) + beta_2_D)
        activation_2_D_S = LeakyReLu(z_2_D_S, 0.2)

        z_3_D_S = []
        for k in range(Batch_Size):
            z_3_D_S.append(np.matmul(Weights_2_to_3_D, activation_2_D_S[k]) + beta_3_D)
        activation_3_D_S = LeakyReLu(z_3_D_S, 0.2)

        z_4_D_S = []
        for k in range(Batch_Size):
            z_4_D_S.append(np.matmul(Weights_3_to_4_D, activation_3_D_S[k]) + beta_4_D)
        activation_4_D_S = LeakyReLu(z_4_D_S, 0.2)

        z_5_D_S = []
        for l in range(Batch_Size):
            z_5_D_S.append(np.dot(Weights_4_to_5_D, activation_4_D_S[l]) + bias_5_D)
        activation_5_D_S = sigmoid(np.array(z_5_D_S))

        ##We start Backpropagating through the Discriminator's Jensen-Shannon Divergence here 

        dJ_D_activation_5_D_S = [0.0] * Batch_Size 
        dJ_D_activation_5_D_G = [0.0] * Batch_Size 

        for i in range(Batch_Size):
            dJ_D_activation_5_D_S[i] = -1/(Batch_Size * activation_5_D_S[i])
            dJ_D_activation_5_D_G[i] = 1/(Batch_Size * (1-activation_5_D_G[i]))

        delta_5_D_S = np.multiply(dJ_D_activation_5_D_S, derivative_sigmoid(z_5_D_S))
        delta_5_D_G = np.multiply(dJ_D_activation_5_D_G, derivative_sigmoid(z_5_D_G))

        ##for j in range(len(Weights_4_to_5_D)):
        dJ_D_Weights_4_to_5_D = np.matmul(delta_5_D_S, activation_4_D_S) + np.matmul(delta_5_D_G, activation_4_D_G)

        dJ_D_bias_4_to_5_D = np.sum(delta_5_D_G) + np.sum(delta_5_D_S)

        dJ_D_activation_4_D_S = np.matmul(delta_5_D_S.reshape(Batch_Size, 1), Weights_4_to_5_D.reshape(157, 1).T)
        dJ_D_activation_4_D_G = np.matmul(delta_5_D_G.reshape(Batch_Size, 1), Weights_4_to_5_D.reshape(157, 1).T)

        delta_4_D_S = np.multiply(dJ_D_activation_4_D_S, derivative_LeakyReLu(z_4_D_S, 0.2))
        delta_4_D_G = np.multiply(dJ_D_activation_4_D_G, derivative_LeakyReLu(z_4_D_G, 0.2))

        dJ_D_Weights_3_to_4_D = np.matmul(delta_4_D_S.T, activation_3_D_S) + np.matmul(delta_4_D_G.T, activation_3_D_G)

        dJ_D_beta_4_D = np.sum(delta_4_D_S) + np.sum(delta_4_D_G)

        dJ_D_activation_3_D_S = np.matmul(delta_4_D_S, Weights_3_to_4_D)

        dJ_D_activation_3_D_G = np.matmul(delta_4_D_G, Weights_3_to_4_D)
        
        delta_3_D_S = np.multiply(dJ_D_activation_3_D_S, derivative_LeakyReLu(z_3_D_S, 0.2))
        delta_3_D_G = np.multiply(dJ_D_activation_3_D_G, derivative_LeakyReLu(z_3_D_G, 0.2))

        dJ_D_Weights_2_to_3_D = np.matmul(delta_3_D_S.T, activation_2_D_S) + np.matmul(delta_3_D_G.T, activation_2_D_G)

        dJ_D_beta_3_D = np.sum(delta_3_D_S) + np.sum(delta_3_D_G)

        dJ_D_activation_2_D_S = np.matmul(delta_3_D_S, Weights_2_to_3_D)

        dJ_D_activation_2_D_G = np.matmul(delta_3_D_G, Weights_2_to_3_D)

        delta_2_D_S = np.multiply(dJ_D_activation_2_D_S, derivative_LeakyReLu(z_2_D_S, 0.2))
        delta_2_D_G = np.multiply(dJ_D_activation_2_D_G, derivative_LeakyReLu(z_2_D_G, 0.2))

        dJ_D_Weights_1_to_2_D = np.matmul(delta_2_D_S.T, activation_1_D_S) + np.matmul(delta_2_D_G.T, activation_1_D_G)

        dJ_D_beta_2_D = np.sum(delta_2_D_S) + np.sum(delta_2_D_G)

        dJ_D_activation_1_D_S = np.matmul(delta_2_D_S, Weights_1_to_2_D)

        dJ_D_activation_1_D_G = np.matmul(delta_2_D_G, Weights_1_to_2_D)

        delta_1_D_S = np.multiply(dJ_D_activation_1_D_S, derivative_LeakyReLu(z_1_D_S, 0.2))
        delta_1_D_G = np.multiply(dJ_D_activation_1_D_G, derivative_LeakyReLu(z_1_D_G, 0.2))

        dJ_D_Weights_0_to_1_D = np.matmul(delta_1_D_G.T, np.array(activation_5_G)) + np.matmul(delta_1_D_S.T, np.array(Transformed_Generation_Training_Set))

        dJ_D_beta_1_D = np.sum(delta_1_D_S) + np.sum(delta_1_D_G)

        ##If we do it, backpropagation through the R1 Regularization term will happen here

        ##OK, this is the backpropagation of the Generator's Jensen Shannon Divergence through the discriminator

        dJ_G_activation_5_D_G = [0.0] * Batch_Size

        for i in range(Batch_Size):
            dJ_G_activation_5_D_G[i] = -1/Batch_Size * 1/activation_5_D_G[i] 

        gamma_5_D = np.multiply(dJ_G_activation_5_D_G, derivative_sigmoid(z_5_D_G))

        dJ_G_activation_4_D_G = np.matmul(gamma_5_D.reshape(Batch_Size, 1), Weights_4_to_5_D.reshape(157, 1).T)

        gamma_4_D = np.multiply(dJ_G_activation_4_D_G, derivative_LeakyReLu(z_4_D_G, 0.2))

        dJ_G_activation_3_D_G = np.matmul(gamma_4_D, Weights_3_to_4_D)

        gamma_3_D = np.multiply(dJ_G_activation_3_D_G, derivative_LeakyReLu(z_3_D_G, 0.2))

        dJ_G_activation_2_D_G = np.matmul(gamma_3_D, Weights_2_to_3_D)

        gamma_2_D = np.multiply(dJ_G_activation_2_D_G, derivative_LeakyReLu(z_2_D_G, 0.2))

        dJ_G_activation_1_D_G = np.matmul(gamma_2_D, Weights_1_to_2_D)

        gamma_1_D = np.multiply(dJ_G_activation_1_D_G, derivative_LeakyReLu(z_1_D_G, 0.2))

        dJ_G_activation_5_G = np.matmul(gamma_1_D, Weights_0_to_1_D)

        ##Ok, this is the backpropagation of the Generator's Jensen Shannon Divergence through the generator  

        gamma_5_G = np.multiply(dJ_G_activation_5_G, np.ones_like(z_5_G) - np.square(np.tanh((z_5_G))))

        dJ_G_Weights_4_to_5_G = np.matmul(gamma_5_G.T, activation_4_G_post)

        dJ_G_bias_5_G = np.sum(gamma_5_G, axis = 0) 

        dJ_G_activation_4_G_post = np.matmul(gamma_5_G, Weights_4_to_5_G)

        (dJ_G_activation_4_G_pre, dJ_G_theta_4_G, dJ_G_beta_4_G) = Batch_Normalization_Derivative(dJ_G_activation_4_G_post, activation_4_G_hat, activation_4_G_pre, mu_4_G, sigma_2_4_G, theta_4_G, beta_4_G, True)

        gamma_4_G = np.multiply(dJ_G_activation_4_G_pre, derivative_LeakyReLu(z_4_G_pre, 0.2))

        dJ_G_Weights_3_to_4_G = np.matmul(gamma_4_G.T, activation_3_G_post)

        dJ_G_activation_3_G_post = np.matmul(gamma_4_G, Weights_3_to_4_G)

        (dJ_G_activation_3_G_pre, dJ_G_theta_3_G, dJ_G_beta_3_G) = Batch_Normalization_Derivative(dJ_G_activation_3_G_post, activation_3_G_hat, activation_3_G_pre, mu_3_G, sigma_2_3_G, theta_3_G, beta_3_G, True)

        gamma_3_G = np.multiply(dJ_G_activation_3_G_pre, derivative_LeakyReLu(z_3_G_pre, 0.2))

        dJ_G_Weights_2_to_3_G = np.matmul(gamma_3_G.T, activation_2_G_post)

        dJ_G_activation_2_G_post = np.matmul(gamma_3_G, Weights_2_to_3_G)

        (dJ_G_activation_2_G_pre, dJ_G_theta_2_G, dJ_G_beta_2_G) = Batch_Normalization_Derivative(dJ_G_activation_2_G_post, activation_2_G_hat, activation_2_G_pre, mu_2_G, sigma_2_2_G, theta_2_G, beta_2_G, True)

        gamma_2_G = np.multiply(dJ_G_activation_2_G_pre, derivative_LeakyReLu(z_2_G_pre, 0.2))

        dJ_G_Weights_1_to_2_G = np.matmul(gamma_2_G.T, activation_1_G_post) 

        dJ_G_activation_1_G_post = np.matmul(gamma_2_G, Weights_1_to_2_G)

        (dJ_G_activation_1_G_pre, dJ_G_theta_1_G, dJ_G_beta_1_G) = Batch_Normalization_Derivative(dJ_G_activation_1_G_post, activation_1_G_hat, activation_1_G_pre, mu_1_G, sigma_2_1_G, theta_1_G, beta_1_G, True)

        gamma_1_G = np.multiply(dJ_G_activation_1_G_pre, derivative_LeakyReLu(z_1_G_pre, 0.2))

        dJ_G_Weights_0_to_1_G = np.dot(gamma_1_G.T, latent_space_random_vector)

        ##OK, and we update all of our weights below: 

        g = dJ_D_Weights_4_to_5_D
        Weights_4_to_5_D_m_matrix = 0.5 * Weights_4_to_5_D_m_matrix + (1-0.5) * np.array(g)
        Weights_4_to_5_D_v_matrix = 0.999 * Weights_4_to_5_D_v_matrix + (1-0.999) * np.square(g)
        m_hat = Weights_4_to_5_D_m_matrix/(1-0.5**r)
        v_hat = Weights_4_to_5_D_v_matrix/(1-0.999**r)
        Weights_4_to_5_D = Weights_4_to_5_D - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))

        g = dJ_D_bias_4_to_5_D
        bias_5_D_m_matrix = 0.5 * bias_5_D_m_matrix + (1-0.5) * np.array(g)
        bias_5_D_v_matrix = 0.999 * bias_5_D_v_matrix + (1-0.999) * np.square(g)
        m_hat = bias_5_D_m_matrix/(1-0.5**r)
        v_hat = bias_5_D_v_matrix/(1-0.999**r)
        bias_5_D = bias_5_D - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))

        g = dJ_D_beta_4_D
        beta_4_D_m_matrix = 0.5 * beta_4_D_m_matrix + (1-0.5) * np.array(g)
        beta_4_D_v_matrix = 0.999 * beta_4_D_v_matrix + (1-0.999) * np.square(g)
        m_hat = beta_4_D_m_matrix/(1-0.5**r)
        v_hat = beta_4_D_v_matrix /(1-0.999**r)
        beta_4_D = beta_4_D - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))

        g = dJ_D_Weights_3_to_4_D
        Weights_3_to_4_D_m_matrix = 0.5 * Weights_3_to_4_D_m_matrix + (1-0.5) * np.array(g)
        Weights_3_to_4_D_v_matrix = 0.999 * Weights_3_to_4_D_v_matrix + (1-0.999) * np.square(g)
        m_hat = Weights_3_to_4_D_m_matrix/(1-0.5**r) 
        v_hat = Weights_3_to_4_D_v_matrix/(1-0.999**r) 
        Weights_3_to_4_D = Weights_3_to_4_D - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))

        g = dJ_D_beta_3_D
        beta_3_D_m_matrix = 0.5 * beta_3_D_m_matrix + (1-0.5) * np.array(g)
        beta_3_D_v_matrix = 0.999 * beta_3_D_v_matrix + (1-0.999) * np.square(g)
        m_hat = beta_3_D_m_matrix/(1-0.5**r)
        v_hat = beta_3_D_v_matrix /(1-0.999**r)
        beta_3_D = beta_3_D - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))

        g = dJ_D_Weights_2_to_3_D
        Weights_2_to_3_D_m_matrix = 0.5 * Weights_2_to_3_D_m_matrix + (1-0.5) * np.array(g)
        Weights_2_to_3_D_v_matrix = 0.999 * Weights_2_to_3_D_v_matrix + (1-0.999) * np.square(g)
        m_hat = Weights_2_to_3_D_m_matrix/(1-0.5**r) 
        v_hat = Weights_2_to_3_D_v_matrix/(1-0.999**r) 
        Weights_2_to_3_D = Weights_2_to_3_D - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))

        g = dJ_D_beta_2_D
        beta_2_D_m_matrix = 0.5 * beta_2_D_m_matrix + (1-0.5) * np.array(g)
        beta_2_D_v_matrix = 0.999 * beta_2_D_v_matrix + (1-0.999) * np.square(g)
        m_hat = beta_2_D_m_matrix/(1-0.5**r)
        v_hat = beta_2_D_v_matrix /(1-0.999**r)
        beta_2_D = beta_2_D - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))

        g = dJ_D_Weights_1_to_2_D
        Weights_1_to_2_D_m_matrix = 0.5 * Weights_1_to_2_D_m_matrix + (1-0.5) * np.array(g)
        Weights_1_to_2_D_v_matrix = 0.999 * Weights_1_to_2_D_v_matrix + (1-0.999) * np.square(g)
        m_hat = Weights_1_to_2_D_m_matrix/(1-0.5**r) 
        v_hat = Weights_1_to_2_D_v_matrix/(1-0.999**r) 
        Weights_1_to_2_D = Weights_1_to_2_D - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))

        g = dJ_D_beta_1_D
        beta_1_D_m_matrix = 0.5 * beta_1_D_m_matrix + (1-0.5) * np.array(g)
        beta_1_D_v_matrix = 0.999 * beta_1_D_v_matrix + (1-0.999) * np.square(g)
        m_hat = beta_1_D_m_matrix/(1-0.5**r)
        v_hat = beta_1_D_v_matrix /(1-0.999**r)
        beta_1_D = beta_1_D - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))

        g = dJ_D_Weights_0_to_1_D
        Weights_0_to_1_D_m_matrix = 0.5 * Weights_0_to_1_D_m_matrix + (1-0.5) * np.array(g)
        Weights_0_to_1_D_v_matrix = 0.999 * Weights_0_to_1_D_v_matrix + (1-0.999) * np.square(g)
        m_hat = Weights_0_to_1_D_m_matrix/(1-0.5**r) 
        v_hat = Weights_0_to_1_D_v_matrix/(1-0.999**r) 
        Weights_0_to_1_D = Weights_0_to_1_D - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))




        g = dJ_G_Weights_4_to_5_G
        Weights_4_to_5_G_m_matrix = 0.5 * Weights_4_to_5_G_m_matrix + (1-0.5) * np.array(g)
        Weights_4_to_5_G_v_matrix = 0.999 * Weights_4_to_5_G_v_matrix + (1-0.999) * np.square(g)
        m_hat = Weights_4_to_5_G_m_matrix/(1-0.5**r)
        v_hat = Weights_4_to_5_G_v_matrix/(1-0.999**r)
        Weights_4_to_5_G = Weights_4_to_5_G - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))

        g = dJ_G_bias_5_G
        bias_5_G_m_matrix = 0.5 * bias_5_G_m_matrix + (1-0.5) * np.array(g)
        bias_5_G_v_matrix = 0.999 * bias_5_G_v_matrix + (1-0.999) * np.square(g)
        m_hat = bias_5_G_m_matrix/(1-0.5**r)
        v_hat = bias_5_G_v_matrix/(1-0.999**r)
        bias_5_G = bias_5_G - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))

        g = dJ_G_theta_4_G
        theta_4_G_m_matrix = 0.5 * theta_4_G_m_matrix + (1-0.5) * np.array(g)
        theta_4_G_v_matrix = 0.999 * theta_4_G_v_matrix + (1-0.999) * np.square(g)
        m_hat = theta_4_G_m_matrix/(1-0.5**r)
        v_hat = theta_4_G_v_matrix/(1-0.999**r)
        theta_4_G = theta_4_G - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))

        g = dJ_G_beta_4_G
        beta_4_G_m_matrix = 0.5 * beta_4_G_m_matrix + (1-0.5) * np.array(g)
        beta_4_G_v_matrix = 0.999 * beta_4_G_v_matrix + (1-0.999) * np.square(g)
        m_hat = beta_4_G_m_matrix/(1-0.5**r)
        v_hat = beta_4_G_v_matrix/(1-0.999**r)
        beta_4_G = beta_4_G - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))

        g = dJ_G_Weights_3_to_4_G
        Weights_3_to_4_G_m_matrix = 0.5 * Weights_3_to_4_G_m_matrix + (1-0.5) * np.array(g)
        Weights_3_to_4_G_v_matrix = 0.999 * Weights_3_to_4_G_v_matrix + (1-0.999) * np.square(g)
        m_hat = Weights_3_to_4_G_m_matrix/(1-0.5**r)
        v_hat = Weights_3_to_4_G_v_matrix/(1-0.999**r)
        Weights_3_to_4_G = Weights_3_to_4_G - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))

        g = dJ_G_theta_3_G
        theta_3_G_m_matrix = 0.5 * theta_3_G_m_matrix + (1-0.5) * np.array(g)
        theta_3_G_v_matrix = 0.999 * theta_3_G_v_matrix + (1-0.999) * np.square(g)
        m_hat = theta_3_G_m_matrix/(1-0.5**r)
        v_hat = theta_3_G_v_matrix/(1-0.999**r)
        theta_3_G = theta_3_G - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))

        g = dJ_G_beta_3_G
        beta_3_G_m_matrix = 0.5 * beta_3_G_m_matrix + (1-0.5) * np.array(g)
        beta_3_G_v_matrix = 0.999 * beta_3_G_v_matrix + (1-0.999) * np.square(g)
        m_hat = beta_3_G_m_matrix/(1-0.5**r)
        v_hat = beta_3_G_v_matrix/(1-0.999**r)
        beta_3_G = beta_3_G - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))

        g = dJ_G_Weights_2_to_3_G
        Weights_2_to_3_G_m_matrix = 0.5 * Weights_2_to_3_G_m_matrix + (1-0.5) * np.array(g)
        Weights_2_to_3_G_v_matrix = 0.999 * Weights_2_to_3_G_v_matrix + (1-0.999) * np.square(g)
        m_hat = Weights_2_to_3_G_m_matrix/(1-0.5**r)
        v_hat = Weights_2_to_3_G_v_matrix/(1-0.999**r)
        Weights_2_to_3_G = Weights_2_to_3_G - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))

        g = dJ_G_theta_2_G 
        theta_2_G_m_matrix = 0.5 * theta_2_G_m_matrix + (1-0.5) * np.array(g)
        theta_2_G_v_matrix = 0.999 * theta_2_G_v_matrix + (1-0.999) * np.square(g)
        m_hat = theta_2_G_m_matrix/(1-0.5**r)
        v_hat = theta_2_G_v_matrix/(1-0.999**r)
        theta_2_G = theta_2_G - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))

        g = dJ_G_beta_2_G 
        beta_2_G_m_matrix = 0.5 * beta_2_G_m_matrix + (1-0.5) * np.array(g)
        beta_2_G_v_matrix = 0.999 * beta_2_G_v_matrix + (1-0.999) * np.square(g)
        m_hat = beta_2_G_m_matrix/(1-0.5**r)
        v_hat = beta_2_G_v_matrix/(1-0.999**r)
        beta_2_G = beta_2_G - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))

        g = dJ_G_Weights_1_to_2_G 
        Weights_1_to_2_G_m_matrix = 0.5 * Weights_1_to_2_G_m_matrix + (1-0.5) * np.array(g)
        Weights_1_to_2_G_v_matrix = 0.999 * Weights_1_to_2_G_v_matrix + (1-0.999) * np.square(g)
        m_hat = Weights_1_to_2_G_m_matrix/(1-0.5**r)
        v_hat = Weights_1_to_2_G_v_matrix/(1-0.999**r)
        Weights_1_to_2_G = Weights_1_to_2_G - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))

        g = dJ_G_theta_1_G 
        theta_1_G_m_matrix = 0.5 * theta_1_G_m_matrix + (1-0.5) * np.array(g)
        theta_1_G_v_matrix = 0.999 * theta_1_G_v_matrix + (1-0.999) * np.square(g)
        m_hat = theta_1_G_m_matrix/(1-0.5**r)
        v_hat = theta_1_G_v_matrix/(1-0.999**r)
        theta_1_G = theta_1_G - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))

        g = dJ_G_beta_1_G 
        beta_1_G_m_matrix = 0.5 * beta_1_G_m_matrix + (1-0.5) * np.array(g)
        beta_1_G_v_matrix = 0.999 * beta_1_G_v_matrix + (1-0.999) * np.square(g)
        m_hat = beta_1_G_m_matrix/(1-0.5**r)
        v_hat = beta_1_G_v_matrix/(1-0.999**r)
        beta_1_G = beta_1_G - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))

        g = dJ_G_Weights_0_to_1_G 
        Weights_0_to_1_G_m_matrix = 0.5 * Weights_0_to_1_G_m_matrix + (1-0.5) * np.array(g)
        Weights_0_to_1_G_v_matrix = 0.999 * Weights_0_to_1_G_v_matrix + (1-0.999) * np.square(g)
        m_hat = Weights_0_to_1_G_m_matrix/(1-0.5**r)
        v_hat = Weights_0_to_1_G_v_matrix/(1-0.999**r)
        Weights_0_to_1_G = Weights_0_to_1_G - 0.0002 * np.divide(m_hat, np.sqrt(v_hat) + 10e-8 * np.ones_like(v_hat))

print("Ok, let's see if it works!")

##Our 5's will (hopefully) be generated here  

latent_space_random_vector = np.random.normal(0, 1, 100*Batch_Size).reshape(Batch_Size, 100)

z_1_G_pre = [] 
for i in range(Batch_Size):
    z_1_G_pre.append(np.matmul(Weights_0_to_1_G, latent_space_random_vector[i]))
activation_1_G_pre = LeakyReLu(z_1_G_pre, 0.2)
activation_1_G_post = test_Batch_Normalization(activation_1_G_pre, mu_1_G_true, sigma_2_1_G_true ,theta_1_G, beta_1_G)

z_2_G_pre = []
for j in range(Batch_Size):
    z_2_G_pre.append(np.matmul(Weights_1_to_2_G, activation_1_G_post[j]))
activation_2_G_pre = LeakyReLu(z_2_G_pre, 0.2)
activation_2_G_post = test_Batch_Normalization(activation_2_G_pre, mu_2_G_true, sigma_2_2_G_true ,theta_2_G, beta_2_G)

z_3_G_pre = []
for k in range(Batch_Size):
    z_3_G_pre.append(np.matmul(Weights_2_to_3_G, activation_2_G_post[k]))
activation_3_G_pre = LeakyReLu(z_3_G_pre, 0.2)
activation_3_G_post = test_Batch_Normalization(activation_3_G_pre, mu_3_G_true, sigma_2_3_G_true ,theta_3_G, beta_3_G)

z_4_G_pre = []
for k in range(Batch_Size):
    z_4_G_pre.append(np.matmul(Weights_3_to_4_G, activation_3_G_post[k]))
activation_4_G_pre = LeakyReLu(z_4_G_pre, 0.2)
activation_4_G_post = test_Batch_Normalization(activation_4_G_pre, mu_4_G_true, sigma_2_4_G_true ,theta_4_G, beta_4_G)

z_5_G = []
for l in range(Batch_Size):
    z_5_G.append(np.matmul(Weights_4_to_5_G, activation_4_G_post[l]) + bias_5_G)
activation_5_G = np.tanh(z_5_G) 

##We will feed our generated 5's through the discriminator to test its strength 

z_1_D_G = []
for i in range(Batch_Size):
    z_1_D_G.append(np.matmul(Weights_0_to_1_D, activation_5_G[i]) + beta_1_D)
activation_1_D_G = LeakyReLu(z_1_D_G, 0.2)

z_2_D_G = []
for j in range(Batch_Size):
    z_2_D_G.append(np.matmul(Weights_1_to_2_D, activation_1_D_G[j]) + beta_2_D)
activation_2_D_G = LeakyReLu(z_2_D_G, 0.2)

z_3_D_G = []
for k in range(Batch_Size):
    z_3_D_G.append(np.matmul(Weights_2_to_3_D, activation_2_D_G[k]) + beta_3_D)
activation_3_D_G = LeakyReLu(z_3_D_G, 0.2)

z_4_D_G = []
for k in range(Batch_Size):
    z_4_D_G.append(np.matmul(Weights_3_to_4_D, activation_3_D_G[k]) + beta_4_D)
activation_4_D_G = LeakyReLu(z_4_D_G, 0.2)

z_5_D_G = []
for l in range(Batch_Size):
    z_5_D_G.append(np.dot(Weights_4_to_5_D, activation_4_D_G[l]) + bias_5_D)
activation_5_D_G = sigmoid(np.array(z_5_D_G))

print(activation_5_D_G)

##We will print our 5's here

activation_5_G = 128.5 * (activation_5_G + np.ones_like(activation_5_G)) - np.ones_like(activation_5_G)

activation_5_G = activation_5_G.reshape(Batch_Size, 28, 28).astype(int)

for i in range(len(activation_5_G)):
    print('sample number', i)
    print('\n'.join([''.join(['{:4}'.format(item) for item in row]) 
        for row in activation_5_G[i]])) 

Ok, let's see if it works!
[0.57619901 0.10761238 0.46161358 0.83508359 0.98546507 0.36543346
 0.19086326 0.83896288 0.06667828 0.26498958 0.15906687 0.17056946
 0.631076   0.38248578 0.21274816 0.60036576 0.50660826 0.76447298
 0.63356963 0.70783068 0.2531468  0.91295062 0.40688152 0.70931234
 0.4937134  0.66684228 0.95581063 0.31349888 0.74608788 0.79712072
 0.6245587  0.09622762 0.40484889 0.22707851 0.0041814  0.21521205
 0.35445406 0.26081098 0.14759431 0.65536066 0.35354026 0.31238751
 0.23647432 0.10767576 0.28780939 0.35100706 0.65280833 0.47528379
 0.2708105  0.77336199]
sample number 0
   0   0   0   0   5   0   0   0   4   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   3   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  16   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   3   0   0  

In [21]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist  
import math
from numpy import random

a = np.random.normal(0, 1, 9).reshape(3, 3)
print(a)
print(a * (a > 0) + 0.2 * a * (a < 0))
print( (a > 0) + 0.2 * (a < 0) )


[[-0.67406782 -0.46465523 -1.62109023]
 [ 1.09202083 -0.79533768 -0.63629436]
 [ 0.22329838 -0.7298643  -0.15493082]]
[[-0.13481356 -0.09293105 -0.32421805]
 [ 1.09202083 -0.15906754 -0.12725887]
 [ 0.22329838 -0.14597286 -0.03098616]]
[[0.2 0.2 0.2]
 [1.  0.2 0.2]
 [1.  0.2 0.2]]
